In [1]:
import math
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# Configurações
T = 2
C = 4
n_heads = 2
head_dim = C // n_heads

# Entrada: 2 tokens com embedding 4
x = torch.tensor([
    [1.0, 2.0, 3.0, 4.0],  # Token 0
    [5.0, 6.0, 7.0, 8.0]   # Token 1
])  # (T=2, C=4)

print("Input x:")
print(x)


Input x:
tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]])


In [3]:
# Criação manual de qkv_proj
# Vamos criar uma camada Linear manual que apenas multiplica por 0.1, 0.2, 0.3
qkv_proj_weight = torch.tensor([
    [0.1, 0.2, 0.3, 0.4],   # Para q0
    [0.5, 0.6, 0.7, 0.8],   # Para q1
    [0.9, 1.0, 1.1, 1.2],   # Para q2
    [1.3, 1.4, 1.5, 1.6],   # Para q3

    [0.1, 0.2, 0.3, 0.4],   # Para k0
    [0.5, 0.6, 0.7, 0.8],   # Para k1
    [0.9, 1.0, 1.1, 1.2],   # Para k2
    [1.3, 1.4, 1.5, 1.6],   # Para k3

    [0.1, 0.2, 0.3, 0.4],   # Para v0
    [0.5, 0.6, 0.7, 0.8],   # Para v1
    [0.9, 1.0, 1.1, 1.2],   # Para v2
    [1.3, 1.4, 1.5, 1.6]    # Para v3
], dtype=torch.float32)  # shape: (3C=12, C=4)

qkv_proj_bias = torch.zeros(12)  # Sem bias para simplificar

# Para out_proj
out_proj_weight = torch.eye(C)  # Identidade (não muda nada)
out_proj_bias = torch.zeros(C)


In [4]:
# Simulando a projeção QKV
qkv = x @ qkv_proj_weight.t() + qkv_proj_bias  # (T, 12)

print("\nqkv:")
print(qkv)

# Separando q, k, v
q, k, v = qkv.chunk(3, dim=1)  # Cada um (T, C)
print("\nq:")
print(q)
print("\nk:")
print(k)
print("\nv:")
print(v)



qkv:
tensor([[ 3.0000,  7.0000, 11.0000, 15.0000,  3.0000,  7.0000, 11.0000, 15.0000,
          3.0000,  7.0000, 11.0000, 15.0000],
        [ 7.0000, 17.4000, 27.8000, 38.2000,  7.0000, 17.4000, 27.8000, 38.2000,
          7.0000, 17.4000, 27.8000, 38.2000]])

q:
tensor([[ 3.0000,  7.0000, 11.0000, 15.0000],
        [ 7.0000, 17.4000, 27.8000, 38.2000]])

k:
tensor([[ 3.0000,  7.0000, 11.0000, 15.0000],
        [ 7.0000, 17.4000, 27.8000, 38.2000]])

v:
tensor([[ 3.0000,  7.0000, 11.0000, 15.0000],
        [ 7.0000, 17.4000, 27.8000, 38.2000]])


In [5]:
att = (q @ k.transpose(-2, -1)) / math.sqrt(head_dim)
# (n_heads, T, T)


In [9]:
k.transpose(-2, -1)

tensor([[ 3.0000,  7.0000],
        [ 7.0000, 17.4000],
        [11.0000, 27.8000],
        [15.0000, 38.2000]])

In [6]:
print(att)

tensor([[ 285.6711,  722.3803],
        [ 722.3803, 1827.0510]])


In [11]:
# Criar máscara causal
mask = torch.tril(torch.ones(T, T)).view(1, 1, T, T)  # (1,1,T,T)
print(mask)

# Aplicar
att = att.masked_fill(mask[:, :, :T, :T] == 0, float('-inf'))

print("\nAttention Scores (depois da máscara):")
print(att)


tensor([[[[1., 0.],
          [1., 1.]]]])

Attention Scores (depois da máscara):
tensor([[[[ 285.6711,      -inf],
          [ 722.3803, 1827.0510]]]])


In [12]:
att = torch.softmax(att, dim=-1)

print("\nAttention Scores (depois do Softmax):")
print(att)



Attention Scores (depois do Softmax):
tensor([[[[1., 0.],
          [0., 1.]]]])


In [13]:
y = att @ v  # (n_heads, T, head_dim)

print("\nOutput das heads (y):")
print(y)



Output das heads (y):
tensor([[[[ 3.0000,  7.0000, 11.0000, 15.0000],
          [ 7.0000, 17.4000, 27.8000, 38.2000]]]])


In [15]:
y = y.transpose(0, 1).contiguous().view(T, C)  # (T, C)

print("\nOutput concatenado (após juntar heads):")
print(y)



Output concatenado (após juntar heads):
tensor([[ 3.0000,  7.0000, 11.0000, 15.0000],
        [ 7.0000, 17.4000, 27.8000, 38.2000]])


In [16]:
# Aplicar projeção final (vamos usar identidade)
out_proj_weight = torch.eye(C)
out_proj_bias = torch.zeros(C)

y = y @ out_proj_weight.t() + out_proj_bias

print("\nSaída final depois da projeção:")
print(y)



Saída final depois da projeção:
tensor([[ 3.0000,  7.0000, 11.0000, 15.0000],
        [ 7.0000, 17.4000, 27.8000, 38.2000]])
